# Expectation-maximization algorithm

In this assignment, we will derive and implement formulas for Gaussian Mixture Model — one of the most commonly used methods for performing soft clustering of the data. 

### Installation

We will need ```numpy```, ```scikit-learn```, ```matplotlib``` libraries for this assignment

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from grader import Grader
%matplotlib inline

### Grading
We will create a grader instance below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to the platform only after running submitting function in the last part of this assignment. If you want to make a partial submission, you can run that cell anytime you want.

In [ ]:
grader = Grader()

## Implementing EM for GMM

For debugging we will use samples from gaussian mixture model with unknown mean, variance and priors. We also added inital values of parameters for grading purposes.

In [ ]:
samples = np.load('samples.npz')
X = samples['data']
pi0 = samples['pi0']
mu0 = samples['mu0']
sigma0 = samples['sigma0']
plt.scatter(X[:, 0], X[:, 1], c='grey', s=30)
plt.axis('equal')
plt.show()

### Reminder

Remember, that EM algorithm is a coordinate descent optimization of variational lower bound $\mathcal{L}(\theta, q) = \int q(T) \log\frac{P(X, T|\theta)}{q(T)}dT\to \max$.

<b>E-step</b>:<br>
$\mathcal{L}(\theta, q) \to \max\limits_{q} \Leftrightarrow \mathcal{KL} [q(T) \,\|\, p(T|X, \theta)] \to \min \limits_{q\in Q} \Rightarrow q(T) = p(T|X, \theta)$<br>
<b>M-step</b>:<br> 
$\mathcal{L}(\theta, q) \to \max\limits_{\theta} \Leftrightarrow \mathbb{E}_{q(T)}\log p(X,T | \theta) \to \max\limits_{\theta}$

For GMM, $\theta$ is a set of parameters that consists of mean vectors $\mu_c$, covariance matrices $\Sigma_c$ and priors $\pi_c$ for each component.

Latent variables $T$ are indices of components to which each data point is assigned. $T_i$ (cluster index for object $i$) is a binary vector with only one active bit in position corresponding to the true component. For example, if we have $C=3$ components and object $i$ lies in first component, $T_i = [1, 0, 0]$.

The joint distribution can be written as follows: $p(T, X \mid \theta) =  \prod\limits_{i=1}^N p(T_i, X_i \mid \theta) = \prod\limits_{i=1}^N \prod\limits_{c=1}^C [\pi_c \mathcal{N}(X_i \mid \mu_c, \Sigma_c)]^{T_{ic}}$.

### E-step
In this step we need to estimate the posterior distribution over the latent variables with fixed values of parameters: $q(T) = p(T|X, \theta)$. We will assume that $T_i$ (cluster index for object $i$) is a binary vector with only one '1' in position corresponding to the true component. To do so we need to compute $\gamma_{ic} = P(T_{ic} = 1 \mid X, \theta)$. Note that $\sum\limits_{c=1}^C\gamma_{ic}=1$.


<b>Important trick 1:</b> It is important to avoid numerical errors. At some point you will have to compute the formula of the following form: $\frac{e^{x_i}}{\sum_j e^{x_j}}$. When you compute exponents of large numbers, you get huge numerical errors (some numbers will simply become infinity). You can avoid this by dividing numerator and denominator by $e^{\max(x)}$: $\frac{e^{x_i-\max(x)}}{\sum_j e^{x_j - \max(x)}}$. After this transformation maximum value in the denominator will be equal to one. All other terms will contribute smaller values. This trick is called log-sum-exp. So, to compute desired formula you first subtract maximum value from each component in vector $X$ and then compute everything else as before.

<b>Important trick 2:</b> You will probably need to compute formula of the form $A^{-1}x$ at some point. You would normally inverse $A$ and then multiply it by $x$. A bit faster and more numerically accurate way to do this is to solve the equation $Ay = x$. Its solution is $y=A^{-1}x$, but the equation $Ay = x$ can be solved by Gaussian elimination procedure. You can use ```np.linalg.solve``` for this.

<b>Other usefull functions: </b> <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.slogdet.html">```slogdet```</a> and <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html#numpy.linalg.det">```det```</a>

<b>Task 1:</b> Implement E-step for GMM using template below.

In [ ]:
from scipy.stats import multivariate_normal

def E_step(X, pi, mu, sigma):
    """
    Performs E-step on GMM model
    
    Parameters
    -----------
    X : np.array, shape (N, d)
        Data points
    pi : np.array, shape (C,)
        Mixture component weights 
    mu : np.array, shape (C, d)
        Mixture component means
    sigma: (C, d, d)
        Mixture component covariance matrices
    
    Returns
    -------
    gamma : np.array (N, C)
        Probabilities of clusters for objects
    """
    
    N = X.shape[0] # number of objects
    C = pi.shape[0] # number of clusters
    d = mu.shape[1] # dimension of each object
    gamma = np.zeros((N, C)) # distribution q(T)

    # NOTE: Before introducing the latent variable we have p(x, theta)
    #       Then we introduce p(t|theta) = pi_c and p(x|t, theta) = N(x|mu_c, Sigma_c)
    #       We are asked to find p(t|x, theta) 
    #       (which we set equal to q as this maximize the E step [i.e. minimize the the gap] due to KL)
    #       Bayes rule states that
    #       p(t|x, theta) = p(x|t, theta) p(t|theta) / p(x|theta)
    #       gamma_c = p(t|x, theta)
    #       Although we don't know p(x|theta), we know that sum gamma_c = 1, and we can use that to normalize
    #       as p(x|theta) is independent of c
        
    for c in range(C):
        # NOTE: The Gaussian is defined as 1/((2pi)^N |Sigma|)exp(-(1/2)(x-mu)^T Sigma^-1(x-mu))
        # We used signed log determinant for robustness
        sign, logdet = np.linalg.slogdet(sigma[c, ...])
        det = sign * np.exp(logdet)
            
        # NOTE: Subtract all columns in the matrix with the vector
        #       Equivalent to A - xI
        x_minus_mu = X - mu[c, ...]
        
        # Let's define y = Sigma^-1(x-mu)
        # NOTE: We transpose here in order for the dimensions to match 
        #       (we are taking the dot product of the dimensionality of x-mu, not of the number of points)
        y = np.linalg.solve(sigma[c, ...], x_minus_mu.transpose())

        # NOTE: The Gaussian maps from R^d to R^1
        #       y has the shape (d, N)
        #       x_minus_mu has the shape (N, d)
        #       The gaussian is to output something of shape (N,)
        #       As we take a vector with dimension d in and want a scalar, we would like to take the
        #       dot product "element-wise"
        x_minus_mu_dot_y = np.empty(N)
        for i in range(N):
            x_minus_mu_dot_y[i] = x_minus_mu[i, :]@y[:, i]
            
        x_minus_mu_dot_y
        gaussian = (1/(det*(2*np.pi)**N)**0.5)*np.exp(-(1/2)*x_minus_mu_dot_y)

        # NOTE: This is the un-normalized gamma
        gamma[:, c] = gaussian * pi[c]
    
    # NOTE: To normalize we realize that a point must belong to one cluster
    #       Thus the gamma summed over c should equal to one
    # We divide all columns in gamma with the same normalization constant
    # https://stackoverflow.com/questions/1550130/cloning-row-or-column-vectors
    norm = np.tile(gamma.sum(axis=1)[..., np.newaxis], (1,3))
    norm = gamma.sum(axis=1, keepdims=True)

    # Trick to avoid large numerical errors
    max_exp = np.exp(-X.max())
    gamma = (gamma*max_exp)/(norm*max_exp)
    
    return gamma

In [ ]:
gamma = E_step(X, pi0, mu0, sigma0)
grader.submit_e_step(gamma)

### M-step

In M-step we need to maximize $\mathbb{E}_{q(T)}\log p(X,T | \theta)$ with respect to $\theta$. In our model this means that we need to find optimal values of $\pi$, $\mu$, $\Sigma$. To do so, you need to compute the derivatives and 
set them to zero. You should start by deriving formulas for $\mu$ as it is the easiest part. Then move on to $\Sigma$. Here it is crucial to optimize function w.r.t. to $\Lambda = \Sigma^{-1}$ and then inverse obtained result. Finaly, to compute $\pi$, you will need <a href="https://www3.nd.edu/~jstiver/FIN360/Constrained%20Optimization.pdf">Lagrange Multipliers technique</a> to satisfy constraint $\sum\limits_{i=1}^{n}\pi_i = 1$.

<br>
<b>Important note:</b> You will need to compute derivatives of scalars with respect to matrices. To refresh this technique from previous courses, see <a href="https://en.wikipedia.org/wiki/Matrix_calculus"> wiki article</a> about it . Main formulas of matrix derivatives can be found in <a href="http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf">Chapter 2 of The Matrix Cookbook</a>. For example, there you may find that $\frac{\partial}{\partial A}\log |A| = A^{-T}$.


<b>Task 2:</b> Implement M-step for GMM using template below.

### Derivation

We now want to maximize

$$
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\sum_{i=1}^N \sum_{c=1}^C q(t_i = c) \log p(t_i=c, x_i \mid \theta)
$$


(see [derivation of the EM algorithm](https://www.coursera.org/learn/bayesian-methods-in-machine-learning/lecture/Fm3mY/expectation-maximization-algorithm) on how to obtain the $\mathcal{L}$ term and [derivation of the M algorithm](https://www.coursera.org/learn/bayesian-methods-in-machine-learning/lecture/46DgL/m-step-details) on how to obatain the $\mathbb{E}_{q(T)}$ term)

We have that

$$
p(T, X \mid \theta) 
= \prod\limits_{i=1}^N p(t_i, x_i \mid \theta)
= \prod\limits_{i=1}^N \prod\limits_{c=1}^C [\pi_c \mathcal{N}(x_i \mid \mu_c, \Sigma_c)]^{T_{ic}}
= \prod\limits_{i=1}^N \prod\limits_{c=1}^C \left[\pi_c
\frac{\exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma }_c|}}
      \right]^{T_{ic}}
$$

so

$$
\log[p(T, X \mid \theta)]
= \log\left[\prod\limits_{i=1}^N \prod\limits_{c=1}^C \left[
\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}
      \exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      \right]^{T_{ic}}\right]\\
= \sum_{i=1}^N \sum_{c=1}^C \log\left[\left[
\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}
      \exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      \right]^{T_{ic}}\right]\\
= \sum_{i=1}^N \sum_{c=1}^C \log\left[\left[
\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}
      \exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      \right]^{T_{ic}}\right]\\
= \sum_{i=1}^N \sum_{c=1}^C T_{ic}\log\left[
\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}
      \exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      \right]\\
= \sum_{i=1}^N \sum_{c=1}^C T_{ic}\left[
\log\left[\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right]
      +
      \log\left[
      \exp 
      \left(-\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right)
      \right]
      \right]\\
= \sum_{i=1}^N \sum_{c=1}^C T_{ic}\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

which means that

$$
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

#### Derivative w.r.t mean

$$
\nabla_{\boldsymbol {\mu}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\nabla_{\boldsymbol {\mu}}
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
\\
= 
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
      -\frac{1}{2}
            \nabla_{\boldsymbol {\mu}}(\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

We use that

$$\partial (\mathbf{A}\mathbf{B}) = (\partial \mathbf{A})\mathbf{B} + \mathbf{A}(\partial \mathbf{B})$$

So

$$
\nabla_{\boldsymbol {\mu}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
=
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
      -\frac{1}{2}
      \left[
            \left[
            \nabla_{\boldsymbol {\mu}}(\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            \right]
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
            +
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            \left[
            \nabla_{\boldsymbol {\mu}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
            \right]
      \right]      
      \right]\\
=
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
      -\frac{1}{2}
      \left[
            \left[
            \nabla_{\boldsymbol {\mu}}(\mathbf{x} - \boldsymbol{\mu}_c)
            \right]^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
            +
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            \left(
            \left[
            \nabla_{\boldsymbol {\mu}}
             \boldsymbol{\Sigma}_c^{-1}
            \right]
            (\mathbf {x} - \boldsymbol {\mu}_c)
            +
             \boldsymbol{\Sigma}_c^{-1}
            \left[
            \nabla_{\boldsymbol {\mu}}
            (\mathbf {x} - \boldsymbol {\mu}_c)            
            \right)
            \right]
      \right]      
      \right]
$$

We note that $\partial_{\boldsymbol {\mu}_{c=i}} \boldsymbol{\mu}_{c=j} = 1 \iff i=j$, and $0$ elsewhere, and that $T_{ic}=1$ only if a point belongs to $c$, meaning that the sum over $C$ disappears

$$
\nabla_{\boldsymbol {\mu}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
=
\sum_{i=1}^N 
q(t_i=c)
\left[
      -\frac{1}{2}
      \left[
            \mathbb{I}^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
            +
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            \left(
            \mathbf {0}
            +
             \boldsymbol{\Sigma}_c^{-1}
            \mathbb{I}           
            \right)
      \right]      
      \right]\\
=
\sum_{i=1}^N 
q(t_i=c)
\left[
      -\frac{1}{2}
      \left[
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
            +
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
      \right]      
      \right]
$$

We use that $\boldsymbol{\Sigma}_c$ is symmetric, which means that $(A\mathbf{x})^T = \mathbf{x}^TA^T = \mathbf{x}^TA$. As the dot product between a matrix and a vector is a vector, we can add the row and the column vector together, and we get

$$
\nabla_{\boldsymbol {\mu}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
=
\sum_{i=1}^N 
q(t_i=c)T_{ic}
\left[
      -\frac{1}{2}
             2\boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]\\
=
-
\sum_{i=1}^N 
q(t_i=c)T_{ic}
      \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
$$

We get the extremes if we set the derivative equal to $0$

$$
\mathbf{0} = \nabla_{\boldsymbol {\mu}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
\\
-\boldsymbol{\Sigma}_c\mathbf{0}
=
\sum_{i=1}^N 
q(t_i=c)
      \boldsymbol{\Sigma}_c\boldsymbol{\Sigma}_c^{-1}
      (\mathbf {x} - \boldsymbol {\mu}_c)\\
\mathbf{0}
=
\sum_{i=1}^N
q(t_i=c)
      (\mathbf {x} - \boldsymbol {\mu}_c)
\\
\sum_{i=1}^N
q(t_i=c)
\boldsymbol {\mu}_c
=
\sum_{i=1}^N
q(t_i=c)
      \mathbf {x}
\\
\boldsymbol {\mu}_c
=
\frac{
\sum_{i=1}^N
q(t_i=c)
      \mathbf {x}
}{
\sum_{i=1}^N
q(t_i=c)
}
$$

#### Derivative w.r.t probability

$$
\nabla_{\boldsymbol {\pi}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\nabla_{\boldsymbol {\pi}}
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
\\
= 
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\nabla_{\boldsymbol {\pi}}
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\mathbf{0}
      \right]
      \\
= 
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\nabla_{\boldsymbol {\pi}}
\log\pi_c
      -
\nabla_{\boldsymbol {\pi}}
\log\left(\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}\right)
      \right]
$$

Using the same arguments as when maximizing the mean, we get

$$
\nabla_{\boldsymbol {\pi}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\sum_{i=1}^N 
q(t_i=c)
\frac{\mathbf{1}}{\pi_c}
$$

We want to maximize this under the constraint $\sum_{c=1}^C \pi_c = 1$. We can reformulate this to the Lagrange multiplier $\sum_{c=1}^C \pi_c - 1 = 0$. We have that

$$
\nabla_{\boldsymbol {\pi}} \left(\sum_{c=1}^C \pi_c - 1\right) = \mathbf{1}
$$

Thus, the maximum is obtained by solving

$$
\sum_{i=1}^N 
q(t_i=c)
\frac{\mathbf{1}}{\pi_c}
- \sum_{i=1}^N = 0
\\
\sum_{i=1}^N 
q(t_i=c)
\frac{\mathbf{1}}{\pi_c}
 = \sum_{i=1}^N
\\
\sum_{i=1}^N 
q(t_i=c)
 = N\pi_c
\\
\pi_c
=
\frac{\sum_{i=1}^N 
q(t_i=c)}{N}
$$

#### Derivative w.r.t variance

$$
\nabla_{\boldsymbol {\Sigma^{-1}}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
= 
\nabla_{\boldsymbol {\Sigma^{-1}}}
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} | \boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]\\
= 
\nabla_{\boldsymbol {\Sigma^{-1}}}
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} }}\right)
      + \log\left(\frac{1}{\sqrt{|\boldsymbol {\Sigma}_c|}}\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

If we use that $|A|^{-1} = |A^{-1}|$, we get

$$
\nabla_{\boldsymbol {\Sigma^{-1}}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
=
\nabla_{\boldsymbol {\Sigma^{-1}}}
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[
\log\left(\frac{\pi_c
      }{\sqrt{(2\pi)^{k} }}\right)
      + \frac{1}{2}\log\left(\left|\boldsymbol {\Sigma}_c^{-1}\right|\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]\\
=
\sum_{i=1}^N \sum_{c=1}^C 
q(t_i=c)T_{ic}
\left[\mathbf{0}
      + \frac{1}{2}\nabla_{\boldsymbol {\Sigma^{-1}}}\log\left(\left|\boldsymbol {\Sigma}_c^{-1}\right|\right)
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
             \nabla_{\boldsymbol {\Sigma^{-1}}}\boldsymbol{\Sigma}_c^{-1}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

We have that

$$
\frac{\partial \log\left(\left|\boldsymbol {\Sigma}_c^{-1}\right|\right)}{\partial{\Sigma}_c^{-1}}
=
\frac{\partial \log\left(\left|\boldsymbol {\Sigma}_c^{-1}\right|\right)}{\partial\left|\boldsymbol {\Sigma}_c^{-1}\right|}
\frac{\partial \left|\boldsymbol {\Sigma}_c^{-1}\right|}{\partial{\Sigma}_c^{-1}}
$$

Since $\frac{\partial \left|\mathbf{A}\right|}{\partial \mathbf{A}} = \left|\mathbf{A}\right|(\mathbf{A}^{-1})^T$, this yields

$$
\frac{\partial \log\left(\left|\boldsymbol {\Sigma}_c^{-1}\right|\right)}{\partial{\Sigma}_c^{-1}}
=
\frac{1}{\left|\boldsymbol {\Sigma}_c^{-1}\right|}
\left|\boldsymbol {\Sigma}_c^{-1}\right|\left(\boldsymbol{\Sigma}_c\right)^T
=
\boldsymbol{\Sigma}_c
$$

as $\boldsymbol{\Sigma}_c$ is symmetric.

Together with the arguments from above we get

$$
\nabla_{\boldsymbol {\Sigma^{-1}}}
\mathbb{E}_{q(T)}\log p(X,T | \theta)
=\sum_{i=1}^N 
q(t_i=c)
\left[\frac{1}{2}\boldsymbol{\Sigma}_c
      -\frac{1}{2}
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
\\
=
\frac{1}{2}
\left[\sum_{i=1}^N q(t_i=c)\boldsymbol{\Sigma}_c
      -
      \sum_{i=1}^N q(t_i=c)
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
$$

Equating this to $\mathbf{0}$ in order to find the maximum gives

$$
\frac{1}{2}
\left[\sum_{i=1}^N q(t_i=c)\boldsymbol{\Sigma}_c
      -
      \sum_{i=1}^N q(t_i=c)
            (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
            (\mathbf {x} - \boldsymbol {\mu}_c)
      \right]
      = \mathbf{0}
\\
\sum_{i=1}^N q(t_i=c)\boldsymbol{\Sigma}_c
=
\sum_{i=1}^N q(t_i=c)
      (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
      (\mathbf {x} - \boldsymbol {\mu}_c)
\\
\boldsymbol{\Sigma}_c
=
\frac{
\sum_{i=1}^N q(t_i=c)
      (\mathbf{x} - \boldsymbol{\mu}_c)^{\mathrm{T}}
      (\mathbf {x} - \boldsymbol {\mu}_c)
}{\sum_{i=1}^N q(t_i=c)}
$$

In [ ]:
def M_step(X, gamma):
    """
    Performs M-step on GMM model
    Each input is numpy array:
    X: (N x d), data points
    gamma: (N x C), distribution q(T)  
    
    Returns:
    pi: (C)
    mu: (C x d)
    sigma: (C x d x d)
    """
    N = X.shape[0] # number of objects
    C = gamma.shape[1] # number of clusters
    d = X.shape[1] # dimension of each object

    ### YOUR CODE HERE

    return pi, mu, sigma

In [ ]:
gamma = E_step(X, pi0, mu0, sigma0)
pi, mu, sigma = M_step(X, gamma)
grader.submit_m_step(pi, mu, sigma)

### Loss function

Finally, we need some function to track convergence. We will use variational lower bound $\mathcal{L}$ for this purpose. We will stop our EM iterations when $\mathcal{L}$ will saturate. Usually, you will need only about 10-20 iterations to converge. It is also useful to check that this function never decreases during training. If it does, you have a bug in your code.

<b>Task 3:</b> Implement a function that will compute $\mathcal{L}$ using template below.

$$\mathcal{L} = \sum_{n=1}^{N} \sum_{k=1}^{K} \mathbb{E}[z_{n, k}] (\log \pi_k + \log \mathcal{N}(x_n | \mu_k, \sigma_k)) - \sum_{n=1}^{N} \sum_{k=1}^{K} \mathbb{E}[z_{n, k}] \log \mathbb{E}[z_{n, k}]$$

In [ ]:
def compute_vlb(X, pi, mu, sigma, gamma):
    """
    Each input is numpy array:
    X: (N x d), data points
    gamma: (N x C), distribution q(T)  
    pi: (C)
    mu: (C x d)
    sigma: (C x d x d)
    
    Returns value of variational lower bound
    """
    N = X.shape[0] # number of objects
    C = gamma.shape[1] # number of clusters
    d = X.shape[1] # dimension of each object

    ### YOUR CODE HERE

    return loss

In [ ]:
pi, mu, sigma = pi0, mu0, sigma0
gamma = E_step(X, pi, mu, sigma)
pi, mu, sigma = M_step(X, gamma)
loss = compute_vlb(X, pi, mu, sigma, gamma)
grader.submit_VLB(loss)

### Bringing it all together

Now that we have E step, M step and VLB, we can implement training loop. We will start at random values of $\pi$, $\mu$ and $\Sigma$, train until $\mathcal{L}$ stops changing and return the resulting points. We also know that EM algorithm sometimes stops at local optima. To avoid this we should restart algorithm multiple times from different starting positions. Each training trial should stop either when maximum number of iterations is reached or when relative improvement is smaller than given tolerance ($|\frac{\mathcal{L}_i-\mathcal{L}_{i-1}}{\mathcal{L}_{i-1}}| \le \text{rtol}$).

Remember, that values of $\pi$ that you generate must be non-negative and sum up to 1. Also, $\Sigma$ matrices must be symmetric and positive semi-definite. If you don't know how to generate those matrices, you can use $\Sigma=I$ as initialization.

You will also sometimes get numerical errors because of component collapsing. The easiest way to deal with this problems is to simply restart the procedure.

<b>Task 4:</b> Implement training procedure

In [ ]:
def train_EM(X, C, rtol=1e-3, max_iter=100, restarts=10):
    '''
    Starts with random initialization *restarts* times
    Runs optimization until saturation with *rtol* reached
    or *max_iter* iterations were made.
    
    X: (N, d), data points
    C: int, number of clusters
    '''
    N = X.shape[0] # number of objects
    d = X.shape[1] # dimension of each object
    best_loss = None
    best_pi = None
    best_mu = None
    best_sigma = None

    for _ in range(restarts):
        try:
            ### YOUR CODE HERE

        except np.linalg.LinAlgError:
            print("Singular matrix: components collapsed")
            pass

    return best_loss, best_pi, best_mu, best_sigma

In [ ]:
best_loss, best_pi, best_mu, best_sigma = train_EM(X, 3)
grader.submit_EM(best_loss)

If you implemented all the steps correctly, your algorithm should converge in about 20 iterations. Let's plot the clusters to see it. We will assign a cluster label as the most probable cluster index. This can be found using matrix $\gamma$ computed on last E-step. 


In [ ]:
gamma = E_step(X, best_pi, best_mu, best_sigma)
labels = gamma.argmax(1)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=30)
plt.axis('equal')
plt.show()

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and your token into variables below. You can generate the token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = # EMAIL HERE
STUDENT_TOKEN = # TOKEN HERE
grader.status()

If you want to submit these answers, run cell below

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)